<a href="https://colab.research.google.com/github/haticebaydemir/AG_News_Classification/blob/main/AG_News_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proje: AG News Sınıflandırma

Hedef: AG News veri seti kullanarak haber metinlerini 4 ana kategoriye (World, Sports, Business, Science/Technology) sınıflandırmak.

# Kütüphaneler

Kullanılan Araçlar ve Kütüphaneler:

Hugging Face Transformers: Önceden eğitilmiş dil modellerini kullanmak için.

PyTorch: Model eğitimi ve optimizasyonu için.

Datasets: Veri setlerini kolayca yönetmek için.

In [ ]:
!pip install transformers torch datasets
# Hugging Face’in transformers kütüphanesi

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

DistilBertTokenizer: DistilBERT modeline uygun şekilde metinleri tokenize etmek için kullanılır.

DistilBertForSequenceClassification: Metin sınıflandırması yapmak için DistilBERT modelini kullanır.

# Veri Setini Yükleme ve İnceleme

 'AG News' veri setini kullanacağız, çünkü bu veri seti haber başlıklarını dört farklı kategoriye ayırır: iş, spor, bilim ve dünya.

In [ ]:
# AG News veri setini yükle
dataset = load_dataset("ag_news", split='train[:1000]')  # İlk 1000 örneği al

# Veri setinin örneklerini inceleyin
for example in dataset:
    print(f"Label: {example['label']}, Text: {example['text']}")
    break  # İlk örneği görüntüle, fazla veri dökümünü önler

Label: 2, Text: Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.


# Veriyi Tokenizasyon

Tokenizasyon: Modelin anlayabileceği sayısal veriler üretmek için metinler tokenize edilir. Bu işlem, metinleri kelime parçacıkları veya token'lara böler.

Bu adımda, metinleri DistilBERT modeline uygun hale getireceğiz.


In [ ]:
# Tokenizer Yükleme
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased') #Bu satır, DistilBERT modelinin önceden eğitilmiş tokenizer'ını yükler. Tokenizer, metinleri modelin anlayacağı forma dönüştürür.

# Tokenizasyon işlemi
def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True, max_length=128)

# Tokenizasyon
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Sütunları yeniden adlandırma ve formatlama
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Eğitim ve test veri setlerine bölme
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']


# Model Seçimi ve Eğitim

Modeli Eğitme: Eğitim verileri üzerinde model eğitilir. Eğitim sırasında model, her haber metnini doğru sınıfa atamayı öğrenir.

In [ ]:
# Modeli Yükleme
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)

# Eğitim argümanları
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Trainer Nesnesini Oluşturma
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Modeli Eğitme
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,1.370800
20,1.375600
30,1.346000
40,1.337400
50,1.323300
60,1.265400
70,1.196200
80,1.209600
90,1.111700
100,1.075900


TrainOutput(global_step=300, training_loss=0.7708656521638234, metrics={'train_runtime': 1858.0543, 'train_samples_per_second': 1.292, 'train_steps_per_second': 0.161, 'total_flos': 79483274035200.0, 'train_loss': 0.7708656521638234, 'epoch': 3.0})

# Modeli Değerlendirme

In [ ]:
# Modeli değerlendirme
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.36997750401496887, 'eval_runtime': 43.9716, 'eval_samples_per_second': 4.548, 'eval_steps_per_second': 0.569, 'epoch': 3.0}


Değerlendirme: Modelin performansı test veri seti üzerinde değerlendirilir ve sonuçlar analiz edilir.

# Tahmin Yapma

In [ ]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions.item()

# Örnek tahmin yapma
sample_text = "Microsoft announces new AI technologies"
predicted_label = predict(sample_text)
print(f"Predicted label: {predicted_label}")

Predicted label: 3


In [ ]:
# Tahmin yapma fonksiyonu
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions.item()

# Test için örnek metinler
sample_texts = [
    "Microsoft announces new AI technologies",
    "The latest sports news and scores",
    "Stock market crashes amid economic uncertainty",
    "New discoveries in quantum physics"
]

# Etiketler
labels = ["World", "Sports", "Business", "Science"]

# Tahminleri yapma ve yazdırma
print("\nSample Predictions:")
for text in sample_texts:
    predicted_label = predict(text)
    predicted_label_name = labels[predicted_label]
    print(f"Text: {text}")
    print(f"Predicted Label: {predicted_label_name}\n")



Sample Predictions:
Text: Microsoft announces new AI technologies
Predicted Label: Science

Text: The latest sports news and scores
Predicted Label: Science

Text: Stock market crashes amid economic uncertainty
Predicted Label: Business

Text: New discoveries in quantum physics
Predicted Label: Science



Bu kod, AG News veri seti kullanarak metin sınıflandırma işlemini gerçekleştirir. Kodun temel adımları veri yükleme, tokenizasyon, model oluşturma, eğitim, değerlendirme ve tahmin yapmayı içerir. Eğitim süreci boyunca modeli eğitir, değerlendirme sonuçlarını alır ve bazı örnek metinler üzerinde tahminler yaparak modelin performansını gösterir.